In [11]:
from asf_hyp3 import API
import pandas as pd
from src.api_functions import hyp3_login, grab_subscription
from pprint import pprint
from datetime import date, datetime
import re

import dataclasses as dc
import numpy as np
from dataclasses import field, asdict
from shapely import wkt
from shapely.geometry import Polygon

In [24]:
@dc.dataclass()
class Product:
    name: str
    granule: str = None
    url: str = None
    shape: Polygon = None
    start: datetime = None
    end: datetime = None
        
    def __post_init__(self):
        self.product_time_regex = re.compile(
                r"S.*1SDV_(?P<start_year>\d{4})(?P<start_month>\d{2})(?P<start_day>\d{2})T(?P<start_hour>\d{2})("
                r"?P<start_minute>\d{2})(?P<start_second>\d{2})_(?P<end_year>\d{4})(?P<end_month>\d{2})(?P<end_day>\d{2})T("
                r"?P<end_hour>\d{2})(?P<end_minute>\d{2})(?P<end_second>\d{2})_*")
#         self.start = make_start(name)
#         self.end = make_end(name)
        self.start = self.make_start(self.granule)
        self.end = self.make_end(self.granule)
        
        

        
    def make_start(self,product_name) -> datetime:
        
        regex_match = re.match(self.product_time_regex, product_name)
        time_dict = regex_match.groupdict()
        for k, v in time_dict.items():
            time_dict[k] = int(v)

        return datetime(time_dict["start_year"], time_dict["start_month"], time_dict["start_day"],
                        time_dict["start_hour"], time_dict["start_minute"], time_dict["start_second"])



    def make_end(self, product_name) -> datetime:

        regex_match = re.match(self.product_time_regex, product_name)
        time_dict = regex_match.groupdict()
        for k, v in time_dict.items():
            time_dict[k] = int(v)

        return datetime(time_dict["end_year"], time_dict["end_month"], time_dict["end_day"],
                        time_dict["end_hour"], time_dict["end_minute"], time_dict["end_second"])
    
    
    def to_json(self):
        metadata = asdict(self)
        metadata['start'] = self.start.isoformat()
        metadata['end'] = self.end.isoformat()
        metadata['shape'] = str(self.shape)
        
#         for key in list(metadata):
#             if key is datetime:
#                 metadata[key] = metadata[key].isoformat()
#                 print(f"TEST: {key}= {metadata[key]}")
        return json.dumps(metadata)
        
        
    

In [3]:
def products_dataframe() -> pd.DataFrame:
    
    df = pd.DataFrame(columns=["name", "id", "start", "end"])


In [18]:
def product_time(product_name):
    product_time_regex = re.compile(
        r"S.*1SDV_(?P<start_year>\d{4})(?P<start_month>\d{2})(?P<start_day>\d{2})T(?P<start_hour>\d{2})("
        r"?P<start_minute>\d{2})(?P<start_second>\d{2})_(?P<end_year>\d{4})(?P<end_month>\d{2})(?P<end_day>\d{2})T("
        r"?P<end_hour>\d{2})(?P<end_minute>\d{2})(?P<end_second>\d{2})_*")

    regex_match = re.match(product_time_regex, product_name)
    time_dict = regex_match.groupdict()

    # converts all dates/times values in dictionary from int to string
    for k, v in time_dict.items():
        time_dict[k] = int(v)

    start = datetime(time_dict["start_year"], time_dict["start_month"], time_dict["start_day"],
                     time_dict["start_hour"], time_dict["start_minute"], time_dict["start_second"])

    end = datetime(time_dict["end_year"], time_dict["end_month"], time_dict["end_day"],
                   time_dict["end_hour"], time_dict["end_minute"], time_dict["end_second"])


    return start, end

In [5]:
def product_in_time_bounds(product_name, start, end):
    prod_start, prod_end = product_time(product_name)
    
    return prod_start > start and prod_end < end

In [22]:
def get_sub_products(api, sub_id, start, end):
    
    response = api.get_products(sub_id=sub_id)
    
    products = []
    for product in response:
        if product_in_time_bounds(product['granule'], start, end):
            products.append(Product(product['name'],
                                    product['granule'],
                                    product['url']
                                   )
                           )
    return products
    


In [25]:
start = datetime(year=2019,month=12,day=3,hour=0,minute=0,second=0)
end   = datetime(year=2019,month=12,day=4,hour=0,minute=0,second=0)

api = hyp3_login()
# subscription = grab_subscription(api)
# sub_id = subscription['id']
id = 2810


products = get_sub_products(api, id, start, end)
pprint(products)

# response = api.get_products(sub_id = id)
# pprint(response)
# products_list = [product['granule'] for product in response]

# products = [p for p in products_list if product_in_time_bounds(p, start, end)]

# # products = get_sub_granules(api, sub_id, start, end)

# print(f"product count is {len(products)}")







 login successful!
 Welcome jmherning
[Product(name='S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9.zip', granule='S1A_IW_GRDH_1SDV_20191203T224518_20191203T224543_030190_037348_297D', url='https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9.zip', shape=None, start=datetime.datetime(2019, 12, 3, 22, 45, 18), end=datetime.datetime(2019, 12, 3, 22, 45, 43)),
 Product(name='S1A_IW_20191203T224543_DVP_RTC10_G_gpuned_3A53.zip', granule='S1A_IW_GRDH_1SDV_20191203T224543_20191203T224608_030190_037348_8C67', url='https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_20191203T224543_DVP_RTC10_G_gpuned_3A53.zip', shape=None, start=datetime.datetime(2019, 12, 3, 22, 45, 43), end=datetime.datetime(2019, 12, 3, 22, 46, 8))]
